# Chapter 3.2

## Importing Libraries

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import tqdm
from hfunc import models
from hfunc import metrics

## Set seed

In [3]:
np.random.seed(2020)

## Self-created functions

In [4]:
def estimate_node_importance(model, tester_model, layer_sizes, tol_low, tol_high, x, y):
    
    l, a = model.evaluate(x, y, verbose=0, batch_size=256)
    or_weights = model.get_weights()
    weight_len = len(or_weights) - 3
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    amounts = []
    places = []
    for layer, size in enumerate(layer_sizes):
        num_zeros, num_worse, num_important = (0, 0, 0)
        z = []
        wr = []
        imp = []
        for i in range(size):
            w = copy.deepcopy(or_weights)
            w[weight_len - (2*layer+1)][:,i] = 0
            w[weight_len - 2*layer][i] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
            change = l - nl
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
        amounts.append((num_zeros, num_worse, num_important))
        places.append((z, wr, imp))
    
    return amounts, places

## Single-Layer ANN

### MNIST

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:21<00:00, 42.42s/it]


In [8]:
NI = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI.to_csv('../../../results/single_layer_node_importance_mnist.csv')
NI.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.00000
mean,2.260000,14.840000,110.90000
std,1.337222,7.913718,7.77555
min,0.000000,1.000000,92.00000
25%,1.000000,10.000000,106.00000
50%,2.000000,14.500000,111.00000
75%,3.000000,20.000000,116.00000
max,6.000000,34.000000,125.00000


### Fashion MNIST

In [9]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [10]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [35:14<00:00, 42.28s/it]


In [12]:
NI = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI.to_csv('../../../results/single_layer_node_importance_fmnist.csv')
NI.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,16.060000,19.960000,91.980000
std,3.519102,6.995509,7.571402
min,8.000000,6.000000,77.000000
25%,14.000000,16.000000,86.000000
50%,15.500000,20.000000,92.500000
75%,18.000000,25.750000,96.000000
max,24.000000,36.000000,111.000000


## Multi-layer Perceptron

### MNIST

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [14]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [32, 64, 128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [58:17<00:00, 69.94s/it]


In [16]:
NI_1 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_3 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_1.to_csv('../../../results/multi_layer_1_node_importance_mnist.csv')
NI_2.to_csv('../../../results/multi_layer_2_node_importance_mnist.csv')
NI_3.to_csv('../../../results/multi_layer_3_node_importance_mnist.csv')

In [17]:
NI_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,3.700000,16.860000,107.440000
std,1.606746,6.104565,6.633742
min,0.000000,5.000000,94.000000
25%,2.250000,13.000000,104.000000
50%,4.000000,16.500000,108.000000
75%,5.000000,20.750000,112.000000
max,7.000000,31.000000,120.000000


In [18]:
NI_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.00000
mean,1.960000,9.480000,52.56000
std,1.497753,3.996121,3.87646
min,0.000000,2.000000,43.00000
25%,1.000000,7.000000,50.25000
50%,2.000000,9.000000,53.00000
75%,3.000000,11.750000,55.75000
max,6.000000,19.000000,60.00000


In [19]:
NI_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,1.920000,2.740000,27.340000
std,1.440521,2.038507,2.421081
min,0.000000,0.000000,20.000000
25%,1.000000,1.000000,26.000000
50%,2.000000,2.500000,27.000000
75%,3.000000,4.000000,29.000000
max,5.000000,9.000000,32.000000


### Fashion MNIST

In [20]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [23]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [32, 64, 128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [58:35<00:00, 70.30s/it]


In [25]:
NI_1 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_3 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_1.to_csv('../../../results/multi_layer_1_node_importance_fmnist.csv')
NI_2.to_csv('../../../results/multi_layer_2_node_importance_fmnist.csv')
NI_3.to_csv('../../../results/multi_layer_3_node_importance_fmnist.csv')

In [26]:
NI_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,18.480000,18.560000,90.960000
std,4.031711,6.295966,6.521425
min,12.000000,8.000000,77.000000
25%,15.250000,13.000000,87.000000
50%,18.000000,19.000000,90.000000
75%,21.000000,23.000000,96.000000
max,31.000000,33.000000,107.000000


In [27]:
NI_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,4.880000,11.660000,47.460000
std,2.237345,4.762138,5.019188
min,1.000000,4.000000,35.000000
25%,3.250000,8.250000,44.250000
50%,5.000000,11.500000,47.500000
75%,6.000000,14.000000,51.000000
max,11.000000,27.000000,58.000000


In [28]:
NI_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.560000,3.380000,26.060000
std,1.680136,2.248718,2.721119
min,0.000000,0.000000,20.000000
25%,1.000000,1.250000,25.000000
50%,2.000000,3.500000,26.000000
75%,3.750000,5.000000,28.000000
max,7.000000,8.000000,31.000000


## Convolutional Neural Network

### MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

### Fashion MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)